In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
import piplite
await piplite.install("palettable")
await piplite.install("seaborn")
import VisualizeNN as VisNN
import pyodide
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)})

## ペンギンのデータセット

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/penguin.csv"))
data.head()

実際に測られたペンギンのデータを読み込んだ

- species : どの種のペンギンなのか
- island : どの島に住んでいるのか
- bill_length_mm : くちばしの長さ
- bill_depth_mm : くちばしの深さ
- flipper_length_mm : 羽の長さ
- body_mass_g : 体重
- sex : 性別
- year : 測った年度

In [ ]:
data.iloc[[0,150,300],:]

In [ ]:
sns.scatterplot(data = data, x = "bill_length_mm", y = "flipper_length_mm", hue = "species")

くちばしと羽の長さの散布図

In [ ]:
data = pd.read_csv(pyodide.http.open_url("https://raw.githubusercontent.com/ChungWookyung/kg-jupyterlite-data-analysis/main/content/DATA/penguin.csv"))
data = data.drop("year",axis=1)
data.sex = data.sex.replace({"female":1,"male":0})
data = pd.get_dummies(data, columns=["island","species"])
data.bill_length_mm = (data.bill_length_mm - data.bill_length_mm.mean())/data.bill_length_mm.std()
data.bill_depth_mm = (data.bill_depth_mm - data.bill_depth_mm.mean())/data.bill_depth_mm.std()
data.flipper_length_mm = (data.flipper_length_mm - data.flipper_length_mm.mean())/data.flipper_length_mm.std()
data.body_mass_g = (data.body_mass_g - data.body_mass_g.mean())/data.body_mass_g.std()
data.head()

データを学習しやすいように書き直す
- ダミー変数に書き換える作業
- -1から1に収まるように正規化する作業

### 1 学習データとテストデータを分ける

In [ ]:
train = data.sample(frac=0.8, random_state=1234)
test = data.drop(train.index)
train.head()

In [ ]:
test.head()

266個の学習データtrainと67個のテストデータtestに分けた

### 2 ニューラルネットワークの構築

In [ ]:
X_train = train.iloc[:,np.arange(8)].values
Y_train = train.loc[:,["species_Adelie","species_Chinstrap","species_Gentoo"]].values

まずは、ニューラルネットワークモデルを学習させるためのXとYデータを分けておく

- X_train : ペンギンの基本情報の学習データ
- Y_train : ペンギンの種の学習データ

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(16,8,), alpha=0.5, max_iter=5000, random_state=1234)
model = model.fit(X_train, Y_train)

１階層に16個のニューロンがあり、２階層に8個のニューロンがあるモデルを構築し、学習させた

In [ ]:
model.score(X_train, Y_train)

学習データにおける正確度は100%である

In [ ]:
X_test = test.iloc[:,np.arange(8)].values
Y_test = test.loc[:,["species_Adelie","species_Chinstrap","species_Gentoo"]].values
model.score(X_test, Y_test)

テストデータにおける正確度も100%である

## 3 ニューラルネットワークモデルの図式化

In [ ]:
network_structure = np.hstack(([X_train.shape[1]], np.asarray(model.hidden_layer_sizes), [3]))
network=VisNN.DrawNN(network_structure)
network.draw()

モデルの各ニューロンは上のようにつながっている

In [ ]:
network=VisNN.DrawNN(network_structure, model.coefs_)
network.draw()

学習されたニューロンの連結を図式化した